In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 400)
pd.set_option("display.max_colwidth", None) # None for all of it

from nltk.corpus import stopwords
german_stop_words = stopwords.words('german')
from HanTa import HanoverTagger as ht
from tqdm import tqdm


PARTYCOLORS =  {'AfD': '#59dcff', "CSU":"#525060", "CDU":"black", "CDU/CSU": "black", "SPD":"#e03427", "GRUENE":"#52d157", "DIE LINKE":"#a33ab5", "FDP":"#ffeb3b",}

In [16]:
stc = pd.read_feather("data/sentences_climate.feather")
stc.shape

(17027, 12)

In [17]:
stc[stc.sentence.str.contains("0.000653")].sample()

,level_0,sent_id,speech_id,name,electoral_term,party,role,date,session,sentence_no,sentence_length,sentence
4187,607230,873733,34114,Karsten Hilse,19,AfD,mp,2018-06-28,42,28,91,"Doch auch wenn unsere Emissionen auf nahe null gesenkt würden, wie es die große Transformation vorsieht und wie es als Staatsziel im sogenannten Klimaschutzplan 2050 festgeschrieben ist, und auch nur dann, wenn man dem Dogma der Ideologie von der menschengemachten Erwärmung folgte, würde der deutsche Senkungsbeitrag zwar zur völligen Verarmung des größten Teils unserer dann vermutlich Multikultibevölkerung führen, aber rechnerisch nur zu einer Verringerung des Anstiegs der sogenannten Welttemperatur irgendwann in ferner Zukunft um 0,000653 Grad Celsius führen – ein Wert, der weder messbar noch fühlbar ist, also praktisch nicht vorhanden ist."


In [3]:
# exclude government references
stc = stc[~stc["sentence"].str.contains("regierung|kanzler|minister|koalition|kabinett|präsident|staatssekretär", case=False)]
print(stc.shape)
stc.value_counts("electoral_term")

(15444, 12)


electoral_term
19    10047
18     5397
Name: count, dtype: int64

In [4]:
hannover = ht.HanoverTagger('morphmodel_ger.pgz')
entire_stc = {}
num = 0
for i, party in enumerate(list(PARTYCOLORS.keys())):
    for y in [18,19]:
        num += 1
        all_stc = stc[(stc["party"] == party) & (stc["electoral_term"] == y)].sentence.to_list()
        speech_stcs = " ".join(all_stc)
        speech_words = []
        for word in tqdm(speech_stcs.split(" ")):
            word = word.lower().strip(" ,.:?;!–-")
            if word == "":
                continue

            if word in german_stop_words or len(word) < 3:
                continue
        
            word = str(hannover.analyze(word)[0]).lower()

            speech_words.append(word)
            
        
        entire_stc[num] = [party + "_" + str(y), " ".join(speech_words)]
len(entire_stc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23087/23087 [00:11<00:00, 1980.79it/s]


16

In [12]:
# Create dataframe
entire_stc = pd.DataFrame.from_dict(entire_stc, orient='index')
entire_stc = entire_stc.rename(columns={0:"party", 1:"text"})
partyterm = entire_stc.party.str.split("_", expand=True)
entire_stc["party"] = partyterm[0]
entire_stc["term"] = partyterm[1].astype("int")

TypeError: 'numpy.ndarray' object is not callable

In [15]:
# Save each term as a separate file
for term in list(stc.electoral_term.unique()):
    entire_stc[entire_stc["term"] == term][["party","text"]].reset_index().to_feather(f"data/selected_stncs_{term}_stem_wgov.feather")

In [7]:
#stc.sample(40)

In [8]:
stc[stc["sentence"].str.contains("stattdessen")].shape

(20, 12)